## 0. 모듈 불러와주기
- requests: 원하는 웹페이지에 요청을 보내 html을 받아오는 것
- BeautifulSoup: html을 파싱(실제 실행가능한 형태로 변환) 해주는 모듈
- pandas: 행과 열로 이루어진 데이터프레임 만들기 위해서
- collections : deque라는 자료구조 활용하기 위해서
- re: regex(정규표현식) 사용하기 위해서

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import collections as co
import re

## 1. 이름만 먼저 가져와보자
- data 요청하기
- data의 content 가져오기
- content를 파싱(원하는 형태로 가공)하기 -> lxml이 빠르다
- name 찾기 : soup.find_all

In [6]:
url = "https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?s_poly_cd=&s_dept_cd=&s_dtl_no=&s_elected_method=&s_up_orig_cd=&s_dw_orig_cd=&s_mem_nm=&currentPage=1&movePageNum=&rowPerPage=300"
data = requests.get(url)
print("request success")
html = data.content
soup = BeautifulSoup(html,"lxml")
name_list = soup.find_all("strong")
chi_list = soup.find_all(name = "span", attrs={"class": "chi"})
sun_list = soup.find_all(name = "dd", attrs={"class": "mt"})
region_list = soup.find_all(name = "dd", attrs={"class": "ht"})

region_list[0].text

request success


'경남 창원시성산구'

In [2]:
url = "https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?s_poly_cd=&s_dept_cd=&s_dtl_no=&s_elected_method=&s_up_orig_cd=&s_dw_orig_cd=&s_mem_nm=&currentPage=1&movePageNum=&rowPerPage=300"
data = requests.get(url)
print("request success")
html = data.content
soup = BeautifulSoup(html,"lxml")

request success


In [12]:
name_list = soup.find_all("strong")
chi_list = soup.find_all(name = "span", attrs = {"class": "chi"})
nth_list = soup.find_all(name = "dd", attrs = {"class": "mt"})
region_list = soup.find_all(name = "dd", attrs = {"class": "ht"})

'경남 창원시성산구'

## 2. 나머지도 가져와보자

In [4]:
name_list

[<strong><a href="javascript:jsMemPop('9771230')" title="강기윤의원정보 새창에서 열림">강기윤</a></strong>,
 <strong><a href="javascript:jsMemPop('9771142')" title="강대식의원정보 새창에서 열림">강대식</a></strong>,
 <strong><a href="javascript:jsMemPop('9771174')" title="강득구의원정보 새창에서 열림">강득구</a></strong>,
 <strong><a href="javascript:jsMemPop('9771233')" title="강민국의원정보 새창에서 열림">강민국</a></strong>,
 <strong><a href="javascript:jsMemPop('9771283')" title="강민정의원정보 새창에서 열림">강민정</a></strong>,
 <strong><a href="javascript:jsMemPop('9770933')" title="강병원의원정보 새창에서 열림">강병원</a></strong>,
 <strong><a href="javascript:jsMemPop('9771116')" title="강선우의원정보 새창에서 열림">강선우</a></strong>,
 <strong><a href="javascript:jsMemPop('9771276')" title="강은미의원정보 새창에서 열림">강은미</a></strong>,
 <strong><a href="javascript:jsMemPop('9771168')" title="강준현의원정보 새창에서 열림">강준현</a></strong>,
 <strong><a href="javascript:jsMemPop('9771007')" title="강훈식의원정보 새창에서 열림">강훈식</a></strong>,
 <strong><a href="javascript:jsMemPop('9771109')" title="고민정의원정보 새창에서 열림">고민정</a

## 3. 끌고 온 데이터를 원하는 형식으로 저장해주자
- 빈 큐를 만들어주자 co.deque([])
- for문을 활용해서 빈 큐에 append 시켜주기
- 한자 깔끔하게 만들어주기: 정규표현식을 활용해보자(re.sub으로 공백문자 치환하기)

In [13]:
temp = co.deque([])

for i in range(len(name_list)):
    print(i)
    temp.append({
        "name": name_list[i].text,
        "hanja": re.sub("\s+","",chi_list[i].text),
        "nth": nth_list[i].text,
        "region":region_list[i].text
    })

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
temp

deque([{'name': '강기윤', 'hanja': '(姜起潤)', 'nth': '재선', 'region': '경남 창원시성산구'},
       {'name': '강대식', 'hanja': '(姜大植)', 'nth': '초선', 'region': '대구 동구을'},
       {'name': '강득구', 'hanja': '(姜得求)', 'nth': '초선', 'region': '경기 안양시만안구'},
       {'name': '강민국', 'hanja': '(姜旻局)', 'nth': '초선', 'region': '경남 진주시을'},
       {'name': '강민정', 'hanja': '(姜旼姃)', 'nth': '초선', 'region': '비례대표'},
       {'name': '강병원', 'hanja': '(姜炳遠)', 'nth': '재선', 'region': '서울 은평구을'},
       {'name': '강선우', 'hanja': '(姜仙祐)', 'nth': '초선', 'region': '서울 강서구갑'},
       {'name': '강은미', 'hanja': '(姜恩美)', 'nth': '초선', 'region': '비례대표'},
       {'name': '강준현', 'hanja': '(康準鉉)', 'nth': '초선', 'region': '세종특별자치시을'},
       {'name': '강훈식', 'hanja': '(姜勳植)', 'nth': '재선', 'region': '충남 아산시을'},
       {'name': '고민정', 'hanja': '(高旼廷)', 'nth': '초선', 'region': '서울 광진구을'},
       {'name': '고영인', 'hanja': '(高永寅)', 'nth': '초선', 'region': '경기 안산시단원구갑'},
       {'name': '고용진', 'hanja': '(高榕禛)', 'nth': '재선', 'region': '서울 노원구갑'},
       {'na

In [15]:
df = pd.DataFrame(temp)
df

,name,hanja,nth,region
0,강기윤,(姜起潤),재선,경남 창원시성산구
1,강대식,(姜大植),초선,대구 동구을
2,강득구,(姜得求),초선,경기 안양시만안구
3,강민국,(姜旻局),초선,경남 진주시을
4,강민정,(姜旼姃),초선,비례대표
...,...,...,...,...
290,홍정민,(洪貞敏),초선,경기 고양시병
291,홍준표,(洪準杓),5선,대구 수성구을
292,황보승희,(皇甫承希),초선,부산 중구영도구
293,황운하,(黃雲夏),초선,대전 중구


In [16]:
df.to_csv("df.csv")

## 4. 데이터프레임으로 만들어주기
- pd.DataFrame

In [4]:
df_temp = pd.DataFrame(temp)
df_temp

,name,hanja,nth,region
0,강기윤,(姜起潤),재선,경남 창원시성산구
1,강대식,(姜大植),초선,대구 동구을
2,강득구,(姜得求),초선,경기 안양시만안구
3,강민국,(姜旻局),초선,경남 진주시을
4,강민정,(姜旼姃),초선,비례대표
...,...,...,...,...
290,홍정민,(洪貞敏),초선,경기 고양시병
291,홍준표,(洪準杓),5선,대구 수성구을
292,황보승희,(皇甫承希),초선,부산 중구영도구
293,황운하,(黃雲夏),초선,대전 중구


## 5. csv파일로 저장하기
- (원하는데이터프레임).to_csv("원하는 파일명.csv")

In [30]:
df_temp.to_csv("df.csv")

## (보너스) 정당명도 함께 긁어오기

원리를 생각해보자.

우선 정당코드로 url을 돌리는 for문, 돌릴 때마다 새롭게 list긁기, 그다음에 list 안에 있는 친구들 긁어오기, 위에 했던 대로 돌려주기, 빈 데이터 프레임에 차례로 합치기가 있다.

In [17]:
df = pd.DataFrame()
code = ['101182','101210','101180','101200','101203','101208','101209','101030']
party = ['더불어민주당','국민의힘','정의당','국민의당','열린민주당','기본소득당','시대전환','무소속']

for i in range(len(code)):
    print(i)
    temp = co.deque([])
    url = "https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?s_poly_cd=" +str(code[i])+"&s_dept_cd=&s_dtl_no=&s_elected_method=&s_up_orig_cd=&s_dw_orig_cd=&s_mem_nm=&currentPage=1&movePageNum=&rowPerPage=300"
    data = requests.get(url)
    print("request success")
    
    html = data.content
    soup = BeautifulSoup(html,"lxml")
    name_list = soup.find_all("strong")
    chi_list = soup.find_all(name = "span", attrs={"class": "chi"})
    sun_list = soup.find_all(name = "dd", attrs={"class": "mt"})
    region_list = soup.find_all(name = "dd", attrs={"class": "ht"})
    partyname = party[i]
    
    for k in range(len(name_list)):
        temp.append({
            "name": name_list[k].text,
            "hanja": re.sub("\s+","",chi_list[k].text),
            "nth": sun_list[k].text,
            "region": region_list[k].text,
            "party": partyname
        })
    temp_df = pd.DataFrame(temp)
    df = pd.concat([df,temp_df])
    
    

0
request success
1
request success
2
request success
3
request success
4
request success
5
request success
6
request success
7
request success


In [18]:
df

,name,hanja,nth,region,party
0,강득구,(姜得求),초선,경기 안양시만안구,더불어민주당
1,강병원,(姜炳遠),재선,서울 은평구을,더불어민주당
2,강선우,(姜仙祐),초선,서울 강서구갑,더불어민주당
3,강준현,(康準鉉),초선,세종특별자치시을,더불어민주당
4,강훈식,(姜勳植),재선,충남 아산시을,더불어민주당
...,...,...,...,...,...
2,박병석,(朴炳錫),6선,대전 서구갑,무소속
3,양정숙,(梁貞淑),초선,비례대표,무소속
4,양향자,(梁香子),초선,광주 서구을,무소속
5,윤미향,(尹美香),초선,비례대표,무소속
